In [ ]:

# -------------------------------------------
# 1. IMPORT, DEFINE PATHS, AND LOAD TUNED MODELS
# -------------------------------------------

# Stage 07: Accuracy and runtime calculations for a selected model
#

# The user selects a single model to evaluate accuracy and runtime.
# This module then loads the selected tuned model's dictionary from Stage 06 and:
#   1. Performs exhaustive feature subset analysis to measure an average runtime for the model
#   2. Generates a confusion matrix and detailed accuracy report for the model.
#   3. Saves results to three Excel files for just the selected model:
#        a. A main results Excel file showing accuracy and runtime for each feature combination
#           (this Excel file is used only for its runtime results).
#        b. A per-class accuracy summary file showing how well the model predicted each bean variety.
#        c. A validation report file with the model's confusion matrix and detailed accuracy report.
#    4. Adds the model's confusion matrix to an existing Excel file containing all models' confusion matrices.
# 


# Import libraries
import pandas as pd
import itertools
import joblib
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import time
import gc
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Define this project's file locations.
# This notebook uses a centralized config.py file for all path management.

# Import config paths
import sys
sys.path.append('..')
from config import TUNED_MODELS_DIR, CURATED_DATA_DIR, RESULTS_DIR

curated_data_dir = CURATED_DATA_DIR
results_dir = RESULTS_DIR
tuned_models_dir = TUNED_MODELS_DIR
results_dir.mkdir(parents=True, exist_ok=True)


# Define file paths
data_path = CURATED_DATA_DIR / "DryBean_curated.parquet"    # From Stage 04
models_path = TUNED_MODELS_DIR / "model_dict.joblib"        # From Stage 06

# Load tuned model dictionary from Stage 06
model_dict = joblib.load(models_path)
print(f"Loaded {len(model_dict)} tuned models from Stage 06")
print(f"Data source: {data_path}")
print(f"Results will be saved to: {results_dir}")

✅ Loaded 16 tuned models from Stage 06
📂 Data source: C:\Misc\ml_benchmark\outputs\curated_data\DryBean_curated.parquet
📂 Results will be saved to: C:\Misc\ml_benchmark\outputs\results


In [3]:
# -------------------------------------------
# 2. SELECT MODEL AND FEATURE SETS
# -------------------------------------------

# Import json for loading two required dictionaries saved from Stage 05
import json


# Display available ML models in the order set in MODEL_FAMILIES dictionary
with open(tuned_models_dir / "model_config.json", "r") as f:
    config = json.load(f)
    MODEL_FAMILIES = config['MODEL_FAMILIES']
    model_mapping = config['model_mapping']
    
ordered_models = [model_mapping[short] for family in MODEL_FAMILIES.values() for _, short in family if model_mapping[short] in model_dict]
print("\n Available tuned models, listed in ML algorithm family order:")
for i, model in enumerate(ordered_models, 1):
    print(f"   {i:2d}. {model}")


# ------------- SELECT MODEL TO TEST HERE -------------
# Change this variable to select the model to test
selected_model_name = "AdaBoostClassifier"  # Change this to test different models
# -----------------------------------------------------

if selected_model_name not in model_dict:
    print(f"❌ Error: {selected_model_name} not found in model_dict")
    print(f"Available models: {available_models}")
else:
    model = model_dict[selected_model_name]
    print(f"\n✅ Selected model: {selected_model_name}")
    
    # Feature Selection Parameters
    # ------ ENTER THE MINIMUM FEATURE COUNT HERE -------
    min_features = 14 
    print(f"🔍 Testing feature set combinations from {min_features} to all features")
    # ---------------------------------------------------

    # Display the model's parameters
    params = model.get_params()
    param_items = list(params.items())
    print(f"Model parameters:")
    for i in range(0, len(param_items), 5):
        chunk = param_items[i:i+5]
        param_str = ", ".join([f"{k}={v}" for k, v in chunk])
        print(f"   {param_str}")


 Available tuned models, listed in ML algorithm family order:
    1. DecisionTreeClassifier
    2. RandomForestClassifier
    3. ExtraTreesClassifier
    4. GradientBoostingClassifier
    5. AdaBoostClassifier
    6. XGBClassifier
    7. LogisticRegression
    8. RidgeClassifier
    9. SGDClassifier
   10. Perceptron
   11. SVC
   12. KNeighborsClassifier
   13. GaussianNB
   14. LinearDiscriminantAnalysis
   15. QuadraticDiscriminantAnalysis
   16. MLPClassifier

✅ Selected model: AdaBoostClassifier
🔍 Testing feature set combinations from 14 to all features
Model parameters:
   algorithm=deprecated, estimator=None, learning_rate=1.0, n_estimators=100, random_state=42


In [4]:
# -------------------------------------------
# 3. LOAD AND PREPROCESS DATA
# -------------------------------------------

# Load processed dataset from Stage 04 (scaled and encoded)
df = pd.read_parquet(data_path)
X = df.drop(columns=["label"])  # Features (already scaled/encoded from Stage 04)
y = df["label"]                 # Target (already label-encoded from Stage 04)

# The data was preprocessed in Stage 04, so can be used here
# without additional scaling or encoding.

feature_names = X.columns.tolist()
total_features = len(feature_names)
unique_classes = y.unique()

# Confirm key dataset information. The last print line confirmas that: 
#   1. Target classes are numeric (some models require numeric labels), and
#   2. No classes are missing.
print(f"\n Dataset Information:")
print(f"   Shape: {X.shape}")
print(f"   Total features: {total_features}")
print(f"   Feature names: {feature_names}")
print(f"   Target classes: {sorted(unique_classes)}")
print(f"   Data types: Features={X.dtypes.iloc[0]}, Target={y.dtype}")


 Dataset Information:
   Shape: (13611, 16)
   Total features: 16
   Feature names: ['A', 'P', 'L', 'l', 'K', 'Ec', 'C', 'Ed', 'Ex', 'S', 'R', 'CO', 'SF1', 'SF2', 'SF3', 'SF4']
   Target classes: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]
   Data types: Features=float64, Target=int64


In [ ]:
# -------------------------------------------
# 4. MEASURE AND RECORD A MODEL'S PERFORMANCE 
# -------------------------------------------

# This code cell is the source of each model's runtimes used in the final results.

# This code creates multiple runs on a range of dataset feature subsets to smooth out runtime 
# variability caused by Windows background activity (e.g., system services or scheduled 
# tasks temporarily affecting CPU availability and RAM management). 


# This code cell creates, for each dataset record, all possible feature sets 
# containing 14 features (120), 15 features (16) , and the original 16-feature record (1) = 137 feature sets.
# This code then loops through tests of the selected model 
# on every feature set created for every record.  Each feature set is examined the number of times 
# set by the user in the variable num_runs. Meaning, each feature set can be examined 1, 2, 3,... times.

# The average accuracy and runtime across all three loops of the same feature set are recorded.
# NOTE: Only the runtime results from this code cell are used in the final results. Accuracy results 
# used for the project's report are determined in a later code cell, below.

# Additionally, the code cell measures how well the model predicts each of the 7 bean varieties.

from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

results = []
per_class_data = []  # Each model's accuracy in predicing each bean variety (aka, class) is stored here.

# Three loops taken on each dataset record. Used to possibly offset runtime outliers caused
# by Windows background processes and RAM management interfering with the model's runtime.
num_runs = 3  

# Get unique class names for consistent reporting
class_names = sorted(y.unique())
print(f"Bean varieties to track: {class_names}")

# Calculate expected combinations for progress tracking.
total_combinations = sum(len(list(itertools.combinations(feature_names, r))) 
                         for r in range(min_features, total_features + 1))
print(f"\nStarting exhaustive feature subset analysis...")
print(f"   Total combinations to test: {total_combinations:,}")
print(f"   Feature range: {min_features} to {total_features} features")

# Conduct benchmark: repeat each feature set 3x, then determine average accuracy and runtime
for r in range(min_features, len(feature_names) + 1):
    combos = list(itertools.combinations(feature_names, r))
    print(f"\nTesting {len(combos):,} combinations of {r} features...")

    for combo in tqdm(combos, desc=f"Feature combinations ({r} features)"):
        accs = []
        runtimes = []
        
        # Track per-class accuracy across all runs
        class_accuracy_runs = []

        for run in range(num_runs):
            model = model_dict[selected_model_name]  # fresh model each run
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

            start_time = time.perf_counter()   #----------------START CLOCK FOR RUNIME CALCULATION-----------------
            
            # Collect predictions for per-class analysis
            y_true_all = []
            y_pred_all = []
            
            # Manual cross-validation to capture individual predictions - generally same runtime for all models
            for train_idx, test_idx in cv.split(X[list(combo)], y):
                X_train, X_test = X[list(combo)].iloc[train_idx], X[list(combo)].iloc[test_idx]
                y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
                
                
                # Train model on 80% of dataset - COMPLETELY DIFFERENT for each model, so different runtimes
                model.fit(X_train, y_train)

                # Generate predictions on 20% of dataset - COMPLETELY DIFFERENT for each model, so different
                # runtimes and accuracies
                y_pred = model.predict(X_test)   
                
                # Store for per-class analysis - generally same runtime for all models
                y_true_all.extend(y_test.tolist())
                y_pred_all.extend(y_pred.tolist())
            
            end_time = time.perf_counter()      #----------------STOP CLOCK FOR RUNIME CALCULATION-----------------
            
            # Calculate overall accuracy
            overall_accuracy = sum(yt == yp for yt, yp in zip(y_true_all, y_pred_all)) / len(y_true_all)
            accs.append(overall_accuracy)
            runtimes.append((end_time - start_time) * 1000)  # ms
            
            # Calculate per-class accuracy for this run
            class_accuracies = {}
            for class_name in class_names:
                class_true_indices = [i for i, yt in enumerate(y_true_all) if yt == class_name]
                if class_true_indices:  # Only if class exists in test set
                    class_correct = sum(1 for i in class_true_indices if y_pred_all[i] == class_name)
                    class_accuracies[class_name] = class_correct / len(class_true_indices)
                else:
                    class_accuracies[class_name] = None  # No samples of this class
            
            class_accuracy_runs.append(class_accuracies)

        # Average over num_runs runs
        avg_acc = round(sum(accs) / num_runs, 4)
        avg_rt = round(sum(runtimes) / num_runs, 3)
        
        # Average per-class accuracies over num_run runs
        avg_class_accuracies = {}
        for class_name in class_names:
            class_accs = [run[class_name] for run in class_accuracy_runs if run[class_name] is not None]
            if class_accs:
                avg_class_accuracies[class_name] = round(sum(class_accs) / len(class_accs), 4)
            else:
                avg_class_accuracies[class_name] = None

        # Build output row: feature columns + blank pads + avg metrics
        row = list(combo) + [""] * (total_features - len(combo)) + [avg_acc, avg_rt]
        results.append(row)
        
        # Store per-class data for this feature combination
        per_class_data.append({
            'Feature_Combination': combo,
            'Feature_Count': len(combo),
            'Overall_Accuracy': avg_acc,
            'Runtime_ms': avg_rt,
            **{class_name: avg_class_accuracies[class_name] for class_name in class_names}
        })

print(f"\n✅ Completed testing {len(results):,} feature combinations")
print(f"Per-class data collected for {len(per_class_data)} feature combinations")

Bean varieties to track: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)]

🔄 Starting exhaustive feature subset analysis...
   Total combinations to test: 137
   Feature range: 14 to 16 features

🧪 Testing 120 combinations of 14 features...


Feature combinations (14 features): 100%|██████████| 120/120 [47:07<00:00, 23.56s/it]



🧪 Testing 16 combinations of 15 features...


Feature combinations (15 features): 100%|██████████| 16/16 [06:39<00:00, 24.97s/it]



🧪 Testing 1 combinations of 16 features...


Feature combinations (16 features): 100%|██████████| 1/1 [00:26<00:00, 26.57s/it]


✅ Completed testing 137 feature combinations
Per-class data collected for 137 feature combinations


In [ ]:
# -------------------------------------------
# 5. ASSEMBLE AND STORE RESULTS IN TWO EXCEL FILES
# -------------------------------------------

# Create a results DataFrame with proper column structure.
# Columns: Feature_1, Feature_2, ..., Feature_16, Accuracy, Runtime (ms)
columns = [f"Feature_{i+1}" for i in range(total_features)] + ["Accuracy", "Runtime (ms)"]
results_df = pd.DataFrame(results, columns=columns)

# Sort by accuracy (descending) to see best performing combinations first
results_df_sorted = results_df.sort_values("Accuracy", ascending=False)


# PREPARE AND SAVE FIRST EXCEL FILE: Accuracy and Runtime Results for each feature set
# Generate the Excel output filename following the naming convention:
# benchmark_{model_name}_results_{min_features}_{max_features}.xlsx

model_name_clean = selected_model_name.lower().replace("classifier", "")
output_filename = f"benchmark_{model_name_clean}_results_{min_features}_{total_features}.xlsx"
output_path = RESULTS_DIR / output_filename

# Create and save the Excel file showing accuracy and runtime for each feature combination, for this model.
results_df_sorted.to_excel(output_path, index=False)


# PREAPARE AND SAVE SECOND EXCEL FILE: Per-Class Accuracy Summary for this model
# Create per-class accuracy summary for this model (how well this model predicted the label's value)
class_names = sorted(y.unique())

# Calculate average per-class accuracy across all feature combinations
model_per_class_summary = {'Model': selected_model_name}

for class_name in class_names:
    class_accuracies = [data[class_name] for data in per_class_data if data[class_name] is not None]
    if class_accuracies:
        avg_class_acc = round(sum(class_accuracies) / len(class_accuracies), 4)
        model_per_class_summary[f'Accuracy_{class_name}'] = avg_class_acc
    else:
        model_per_class_summary[f'Accuracy_{class_name}'] = None

# Create per-class DataFrame (single row)
per_class_df = pd.DataFrame([model_per_class_summary])

# Generate per-class Excel filename
per_class_filename = f"benchmark_{model_name_clean}_per_class_{min_features}_{total_features}.xlsx"
per_class_path = RESULTS_DIR / per_class_filename

# Create and then save the Excel file showing this model's accuracy in identifying each bean variety
per_class_df.to_excel(per_class_path, index=False)

# Display summary statistics
best_accuracy = results_df_sorted['Accuracy'].iloc[0]
total_runtime_sec = results_df['Runtime (ms)'].sum() / 1000
avg_runtime_ms = results_df['Runtime (ms)'].mean()

print(f"\nAnalysis Summary:")
print(f"   Model tested: {selected_model_name}")
print(f"   Feature combinations: {len(results):,}")
print(f"   Best accuracy: {best_accuracy:.4f}")
print(f"   Total runtime: {total_runtime_sec:.1f} seconds")
print(f"   Average runtime per combination: {avg_runtime_ms:.1f} ms")
print(f"\nResults saved to: {output_path}")
print(f"Per-class results saved to: {per_class_path}")

# Display top 5 feature combinations
print(f"\nTop 5 Feature Combinations:")
for i, row in results_df_sorted.head().iterrows():
    features_used = [col for col in columns[:-2] if row[col] != ""]
    feature_list = [row[col] for col in features_used]
    print(f"   {row['Accuracy']:.4f} | {len(feature_list)} features | {feature_list}")

# Display per-class accuracy summary
print(f"\nPer-Class Accuracy Summary for {selected_model_name}:")
for class_name in class_names:
    class_acc = model_per_class_summary[f'Accuracy_{class_name}']
    if class_acc is not None:
        print(f"   {class_name}: {class_acc:.4f} ({class_acc*100:.2f}%)")
    else:
        print(f"   {class_name}: No data")


📊 Analysis Summary:
   Model tested: AdaBoostClassifier
   Feature combinations: 137
   Best accuracy: 0.8611
   Total runtime: 1083.9 seconds
   Average runtime per combination: 7911.6 ms

💾 Results saved to: C:\Misc\ml_benchmark\outputs\results\benchmark_adaboost_results_14_16.xlsx
💾 Per-class results saved to: C:\Misc\ml_benchmark\outputs\results\benchmark_adaboost_per_class_14_16.xlsx

Top 5 Feature Combinations:
   0.8611 | 14 features | ['A', 'P', 'l', 'K', 'Ec', 'C', 'Ed', 'Ex', 'S', 'R', 'CO', 'SF1', 'SF3', 'SF4']
   0.8545 | 14 features | ['A', 'L', 'l', 'K', 'Ec', 'C', 'Ed', 'Ex', 'S', 'R', 'CO', 'SF1', 'SF3', 'SF4']
   0.8524 | 14 features | ['P', 'L', 'K', 'Ec', 'C', 'Ed', 'Ex', 'S', 'R', 'CO', 'SF1', 'SF2', 'SF3', 'SF4']
   0.8524 | 14 features | ['A', 'P', 'L', 'K', 'C', 'Ed', 'Ex', 'S', 'R', 'CO', 'SF1', 'SF2', 'SF3', 'SF4']
   0.8524 | 14 features | ['A', 'P', 'L', 'K', 'Ec', 'C', 'Ex', 'S', 'R', 'CO', 'SF1', 'SF2', 'SF3', 'SF4']

Per-Class Accuracy Summary for AdaBoos

Validation Methodology: Single Feature Set Analysis

The validation section below employs a fundamentally different approach compared to the main benchmarking code. While the main analysis iterates through 137 different feature combinations (ranging from 14 to 16 features), in order to determine a more stable estimate of each model's runtime, the validation below focuses exclusively on the complete, original 16-feature dataset to provide a detailed accuracy baseline and confusion matrix analysis.

Key Differences in Validation Approach:
- **Single Feature Set**: Uses only the complete 16-feature dataset rather than testing multiple feature combinations
- **Detailed Confusion Matrix**: Generates a comprehensive confusion matrix showing exactly which bean classes are confused with others
- **Per-Class Accuracy Validation**: Calculates individual accuracy for each of the 7 bean varieties using the same cross-validation methodology as the main code
- **Identical Cross-Validation**: Maintains the same 5-fold stratified cross-validation with random_state=42 for direct comparison with main results
- **Comprehensive Reporting**: Creates detailed text reports and Excel files specifically for validation and model comparison purposes

This validation approach serves to confirm the accuracy calculation methodology used in the main benchmark while providing deeper insights into model performance patterns that may be masked when averaging across 137 different feature combinations.

In [ ]:
# -------------------------------------------
# 6. DETERMINE ACCURACY OF A SELECTED MODEL
# -------------------------------------------

# PURPOSE:
# This code cell creates a confusion matrix and detailed accuracy report for the selected model to:
#   1. Validate the per-class accuracy calculations from the main benchmark (prior Section 4)
#   2. Provide detailed insight into which bean varieties the model predicts well/poorly
#   3. Confirm the overall accuracy methodology by using only the complete, 16-feature, orginal dataset
#   4. Generate a comprehensive text report saved alongside the Excel benchmark files
#
# APPROACH:
# This code cell uses identical cross-validation methodology as the main benchmark, but focuses on:
#   - Only the complete feature set (all 16 features) rather than iterating through other feature subsets
#   - Detailed confusion matrix showing prediction patterns for each bean class
#   - Per-class accuracy breakdown to identify model strengths/weaknesses
#   - Comparison context for interpreting the averaged results in the Excel files
#
# OUTPUTS: 
#   1. Creates a detailed .txt validation report for the selected model in the same directory as Excel results
#   2. Adds the selected model's confusion matrix to an existing Excel file containing all models' confusion matrices
#   3. Creates an Excel file for the selected model with F1-scores and overall accuracy

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from datetime import datetime

print(f"\nVALIDATION: Creating confusion matrix for {selected_model_name}")
print(f"Using all {total_features} features on {len(X)} records")
print(f"This validates the per-class accuracy calculations")

# Use the same cross-validation setup as the main analysis
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Collect all predictions using identical methodology
y_true_all = []
y_pred_all = []

print(f"\nRunning cross-validation...")

# Manual cross-validation (identical to main code)
for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    # Use fresh model instance and train/predict (same as main code)
    validation_model = model_dict[selected_model_name]
    validation_model.fit(X_train, y_train)
    y_pred = validation_model.predict(X_test)
    
    # Store all predictions
    y_true_all.extend(y_test.tolist())
    y_pred_all.extend(y_pred.tolist())

# Create confusion matrix
cm = confusion_matrix(y_true_all, y_pred_all)

# Create readable confusion matrix with class labels
cm_df = pd.DataFrame(cm, 
                     index=[f"True_{cls}" for cls in sorted(y.unique())], 
                     columns=[f"Pred_{cls}" for cls in sorted(y.unique())])

# Calculate per-class accuracy from confusion matrix
validation_per_class = {}
for i, class_name in enumerate(sorted(y.unique())):
    class_total = cm[i, :].sum()  # Total actual samples of this class
    class_correct = cm[i, i]      # Correctly predicted samples
    class_accuracy = class_correct / class_total if class_total > 0 else 0
    validation_per_class[class_name] = class_accuracy

# Calculate overall accuracy
overall_accuracy = sum(y_true_all[i] == y_pred_all[i] for i in range(len(y_true_all))) / len(y_true_all)

# Identify perfect and poor performing classes
perfect_classes = [cls for cls, acc in validation_per_class.items() if acc == 1.0]
poor_classes = [cls for cls, acc in validation_per_class.items() if acc < 0.8]

# Generate validation report filename
model_name_clean = selected_model_name.lower().replace("classifier", "")
validation_filename = f"benchmark_{model_name_clean}_validation_{min_features}_{total_features}.txt"
validation_path = RESULTS_DIR / validation_filename

# Create comprehensive text report
report_content = []
report_content.append("=" * 80)
report_content.append(f"VALIDATION REPORT: {selected_model_name}")
report_content.append("=" * 80)
report_content.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_content.append(f"Dataset: {len(X):,} records, {total_features} features")
report_content.append(f"Validation uses all {total_features} features (complete feature set)")
report_content.append("")

# Confusion Matrix Section
report_content.append("CONFUSION MATRIX")
report_content.append("-" * 40)
report_content.append("Rows = True Bean Classes, Columns = Predicted Bean Classes")
report_content.append(f"Bean Classes: {sorted(y.unique())}")
report_content.append("")
report_content.append(cm_df.to_string())
report_content.append("")

# Per-Class Accuracy Section
report_content.append("PER-CLASS ACCURACY (from confusion matrix)")
report_content.append("-" * 50)
for i, class_name in enumerate(sorted(y.unique())):
    class_total = cm[i, :].sum()
    class_correct = cm[i, i]
    class_accuracy = validation_per_class[class_name]
    report_content.append(f"   Bean Class {class_name}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%) - {class_correct}/{class_total} correct")

report_content.append("")

# Overall Accuracy Section
report_content.append("OVERALL PERFORMANCE")
report_content.append("-" * 30)
report_content.append(f"Overall Accuracy: {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
report_content.append("")

# Performance Classification
if perfect_classes:
    report_content.append(f"Perfect Classification (100%): Bean classes {perfect_classes}")
if poor_classes:
    report_content.append(f"Challenging Classification (<80%): Bean classes {poor_classes}")
if not perfect_classes and not poor_classes:
    report_content.append("All classes achieved 80%+ accuracy, none achieved 100%")

report_content.append("")

# Comparison to Main Results
report_content.append("COMPARISON TO MAIN BENCHMARK RESULTS")
report_content.append("-" * 45)
report_content.append("   • These per-class accuracies are for the complete 16-feature set only")
report_content.append("   • Your main benchmark Excel file shows averages across all 137 feature combinations")
report_content.append("   • Expected: These validation results may differ slightly from main benchmark averages")
report_content.append("   • This validation confirms the accuracy calculation methodology")
report_content.append("")

# Additional Statistics
report_content.append("DETAILED STATISTICS")
report_content.append("-" * 25)
report_content.append(f"Cross-validation folds: 5")
report_content.append(f"Random state: 42 (reproducible results)")
report_content.append(f"Total predictions made: {len(y_true_all):,}")
report_content.append(f"Correct predictions: {sum(y_true_all[i] == y_pred_all[i] for i in range(len(y_true_all))):,}")
report_content.append(f"Incorrect predictions: {len(y_true_all) - sum(y_true_all[i] == y_pred_all[i] for i in range(len(y_true_all))):,}")

# Class distribution
report_content.append("")
report_content.append("CLASS DISTRIBUTION IN VALIDATION")
report_content.append("-" * 35)
for class_name in sorted(y.unique()):
    class_count = y_true_all.count(class_name)
    class_percentage = (class_count / len(y_true_all)) * 100
    report_content.append(f"   Bean Class {class_name}: {class_count:,} samples ({class_percentage:.1f}%)")

report_content.append("")
report_content.append("=" * 80)
report_content.append("END OF VALIDATION REPORT")
report_content.append("=" * 80)

# Write to file
with open(validation_path, 'w') as f:
    f.write('\n'.join(report_content))

# Display summary to console
print(f"\nVALIDATION SUMMARY:")
print(f"   Model: {selected_model_name}")
print(f"   Overall Accuracy: {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
if perfect_classes:
    print(f"   Perfect Classes: {perfect_classes}")
if poor_classes:
    print(f"   ⚠️  Challenging Classes: {poor_classes}")

print(f"\nValidation report saved to: {validation_path}")
print(f"Report includes confusion matrix, per-class accuracy, and detailed statistics")
print(f"\nValidation complete! All results saved to text file.")


# Save confusion matrix to Excel format for consistency analysis
cm_excel_filename = f"confusion_matrices_all_models_{min_features}_{total_features}.xlsx"
cm_excel_path = RESULTS_DIR / cm_excel_filename

# Add model name column to confusion matrix
cm_df_excel = cm_df.copy()
cm_df_excel.insert(0, 'Model', selected_model_name)

# Check if Excel file already exists
if cm_excel_path.exists():
    # File exists - add new sheet to existing workbook
    with pd.ExcelWriter(cm_excel_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        cm_df_excel.to_excel(writer, sheet_name=selected_model_name, index=True)
    print(f"Added {selected_model_name} sheet to existing Excel file: {cm_excel_path}")
else:
    # File doesn't exist - create new workbook with first sheet
    cm_df_excel.to_excel(cm_excel_path, sheet_name=selected_model_name, index=True)
    print(f"Created new Excel file with {selected_model_name} sheet: {cm_excel_path}")


# Calculate and save F1-scores for this model
from sklearn.metrics import f1_score

# Calculate F1-scores using existing predictions from validation
f1_macro = f1_score(y_true_all, y_pred_all, average='macro')
f1_weighted = f1_score(y_true_all, y_pred_all, average='weighted')

# Create F1-scores summary (single row)
f1_summary = {
    'Model': selected_model_name,
    'F1_Macro': round(f1_macro, 4),
    'F1_Weighted': round(f1_weighted, 4),
    'Overall_Accuracy': round(overall_accuracy, 4)
}

# Create F1-scores DataFrame and save to Excel
f1_df = pd.DataFrame([f1_summary])
f1_filename = f"benchmark_{model_name_clean}_f1_scores_{min_features}_{total_features}.xlsx"
f1_path = RESULTS_DIR / f1_filename
f1_df.to_excel(f1_path, index=False)

# Confirmation print with full path
print(f"F1-scores Excel file created and saved: {f1_path}")
print(f"   F1-Macro: {f1_macro:.4f}, F1-Weighted: {f1_weighted:.4f}")    


VALIDATION: Creating confusion matrix for GradientBoostingClassifier
Using all 16 features on 13611 records
This validates the per-class accuracy calculations

Running cross-validation...

VALIDATION SUMMARY:
   Model: GradientBoostingClassifier
   Overall Accuracy: 0.9269 (92.69%)

Validation report saved to: C:\Misc\ml_benchmark\outputs\results\benchmark_gradientboosting_validation_14_16.txt
Report includes confusion matrix, per-class accuracy, and detailed statistics

Validation complete! All results saved to text file.
Added GradientBoostingClassifier sheet to existing Excel file: C:\Misc\ml_benchmark\outputs\results\confusion_matrices_all_models_14_16.xlsx
F1-scores Excel file created and saved: C:\Misc\ml_benchmark\outputs\results\benchmark_gradientboosting_f1_scores_14_16.xlsx
   F1-Macro: 0.9385, F1-Weighted: 0.9269


In [ ]:
# -------------------------------------------
# 7. CONSOLIDATE F1-SCORES FROM ALL MODELS INTO SINGLE EXCEL FILE
# -------------------------------------------

import pandas as pd
from pathlib import Path

# Define search pattern for F1-score files
f1_pattern = f"benchmark_*_f1_scores_{min_features}_{total_features}.xlsx"
f1_files = list(RESULTS_DIR.glob(f1_pattern))    # Uses the .glob() method of the pathlib.Path object, so glob import not needed

# Create consolidated F1-scores Excel file
consolidated_f1_filename = f"f1_scores_all_models_{min_features}_{total_features}.xlsx"
consolidated_f1_path = RESULTS_DIR / consolidated_f1_filename

if f1_files:
    print(f"\nConsolidating F1-scores from {len(f1_files)} models...")
    
    with pd.ExcelWriter(consolidated_f1_path, engine='openpyxl') as writer:
        for f1_file in f1_files:
            # Extract model name from filename
            model_name = f1_file.stem.replace(f"benchmark_", "").replace(f"_f1_scores_{min_features}_{total_features}", "")
            
            # Read the F1-score data
            f1_data = pd.read_excel(f1_file)
            
            # Write to sheet named after model
            f1_data.to_excel(writer, sheet_name=model_name, index=False)
            
            print(f"   Added {model_name} F1-scores to consolidated file")
    
    print(f"\nF1-scores consolidated successfully: {consolidated_f1_path}")
    print(f"   Total sheets: {len(f1_files)}")
    
else:
    print(f"\n⚠️  No F1-score files found matching pattern: {f1_pattern}")


Consolidating F1-scores from 16 models...
   Added adaboost F1-scores to consolidated file
   Added decisiontree F1-scores to consolidated file
   Added extratrees F1-scores to consolidated file
   Added gaussiannb F1-scores to consolidated file
   Added gradientboosting F1-scores to consolidated file
   Added kneighbors F1-scores to consolidated file
   Added lineardiscriminantanalysis F1-scores to consolidated file
   Added logisticregression F1-scores to consolidated file
   Added mlp F1-scores to consolidated file
   Added perceptron F1-scores to consolidated file
   Added quadraticdiscriminantanalysis F1-scores to consolidated file
   Added randomforest F1-scores to consolidated file
   Added ridge F1-scores to consolidated file
   Added sgd F1-scores to consolidated file
   Added svc F1-scores to consolidated file
   Added xgb F1-scores to consolidated file

✅ F1-scores consolidated successfully: C:\Misc\ml_benchmark\outputs\results\f1_scores_all_models_14_16.xlsx
   Total sheet